# Programming with Energy System Description Language

<a href="https://energytransition.gitbook.io/esdl/">Energy System Description Language (ESDL)</a> is a modelling language created for modelling the components in an energy system and their relations between each other. It also includes dynamic information about the system such as profiles, geographic information such as  location and space, and information about costs. ESDL is capable of expressing the dynamic behaviour of components in the energy system. For instance, the power consumption of a neighbourhood. ESDL describes components by their basic functionality (so called Energy Capabilities), these are modelled in five abstract categories: Production, Consumption, Storage, Transport and Conversion. ESDL enables energy modellers to model a complex energy system in a generic way. The language is machine-readable so makers of energy transition calculation tools and GIS applications can support ESDL in order to enforce the interoperability of their products.

This tutorial is a starting guide for programming with ESDL using Python. The first part of the tutorial explains how to set up the environment to program using ESDL, including the necessary packages, scripts, libraries and configurations. 

The second part guides you, step-by-step, through the process of creating an energy system using ESDL using a simple use case. 

The tutorial starts with an idea of a very simple energy system, translates it into a conceptual model, and implements it using ESDL. At the end of this tutorial, an XML file of the created energy system is stored locally (with a .esdl extension) and can be downloaded to explore its structure. Furthermore, the Jupyter Notebook file of the tutorial can also be downloaded to execute and manipulate the code.

## Setting up the environment for programming with ESDL

As ESDL is an ECore model, Python library pyecore and pyecoregen have to be installed in order to read and work with the ESDL model. The tutorial reads profile data from a timeseries database InfluxDB, so the influxdb package also has to be installed.

In [9]:
run_install = False
#Install pyecore and pyecoregen packages to create and build energy systems in Python using Energy System Description Language (ESDL)

import sys

if run_install:
    !{sys.executable} -m pip install pyecore
    !{sys.executable} -m pip install pyecoregen
    !{sys.executable} -m pip install influxdb

The following code is the starting point for loading, creating and generating ESDL energy system models. It does not create a specific instance of an energy system, but rather loads the necessary resources to create or load one. ESDL Ecore file is an XML file describing all the components that can be modeled using ESDL, their attributes and how they are related.

In [10]:
#PyEcoreGen can be used to generate the ESDL-classes from the esdl.ecore model. 
#On the command line (or directly here in the Notebook) do the following:

if run_install:
    ! pyecoregen -e https://raw.githubusercontent.com/EnergyTransition/ESDL/master/esdl/model/esdl.ecore --auto-register-package -o .

To be able to create energy systems using ESDL, a number of libraries has to be loaded. 

- xmlresource.py - A script that parses XMI to XML (explained below)

- edrreader.py - A script to connect to Energy Data Repository (EDR) and get details to retrieve data from the database

- influxdbreader.py - A script to connect to a timeseries database, InfluxDB, and read data profiles; uses edrreader.py 


Currently, PyEcore supports the XMI format for storing models and model instances. As ESDL is currently not using the XMI features, but only the XMLResource in the plugins, you need to register esdl-files with the XMLResource, otherwise the XMI namespace and the XMI version information is added to the ESDL-file when saving, giving errors when loading this file in Eclipse with the ESDL plugins. To use XMLResources you need to download the file xmlresource.py attached at the end of this page and import it in your project (as done in all the examples below).

## Creating an Energy System 

An ESDL energy system (or a number of them) can be created in two ways: programatically through Python code, or by loading it from an existing ESDL file. 

The following sections cover both of these approaches through a simple use case. The first part of the tutorials focuses on programatically creating and manipulating an energy system, and the second part demonstrates how an existing ESDL file can be loaded, and worked with. 

In this tutorial, a simple energy system is created, with one consumer (with electricty demand only) and one producer, in this case a PV park. In ESDL, a consumer is abstracted, and can represent a single household, or aggregated to a level of, for example, a municipality or a region. For the purpose of this tutorial, a municipality consumer is modelled. The consumer and the producer are connected through an electricity network.

The tutorial demonstrates how to create an energy system, its energy assets, the connections between them, and how to read and store profiles for different assets. 

More information about the energy system structure and data types within ESDL can be found on the following links: 

- <a href="https://energytransition.github.io/">ESDL Model Reference Documentation</a>

- <a href="https://energytransition.gitbook.io/esdl/esdl-concepts/energy-system-structure">ESDL Energy System Structure</a>

- <a href="https://energytransition.gitbook.io/esdl/esdl-concepts/data-types">ESDL Data Types</a>

### 1 Programmatically creating and manipulating an Energy System

### 1.1 Creating energy assets 

To create an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a> using ESDL, an object and an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Instance.html">Instance</a> of an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a> have to be created. An <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a> describes an area, therefore, an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Area.html">Area</a> object is created and added to the energy system <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Instance.html">Instance</a>.

The first step in creating the actual elements of an ESDL <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a> is to create its assets. <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Asset.html">Assets</a> are all the physical items in an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a>. <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Asset.html">Assets</a> can have a location, a geometry, commissioning and decommissioning dates, cost information (investment, installation and operation and maintenance costs) etc., and are contained by <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Area.html">Areas</a>. In this tutorial, a consumer with an electricity demand and a PV park are connected through an electricity network. <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/ElectricityDemand.html">ElectricityDemand</a>, <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/PVPark.html">PVPark</a> and <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/ElectricityNetwork.html">ElectricityNetwork</a> are all examples of different <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergyAsset.html">Energy Asset</a>.

<img src="images/assets.png">


In [1]:
from xmlresource import XMLResource
from datetime import datetime
import uuid
import esdl 
from pyecore.resources import ResourceSet, URI

rset = ResourceSet()

#Create an energy system
energy_system = esdl.EnergySystem(name="Nederland ES")
energy_system_instance = esdl.Instance(name="NL")

#Instantiate the created energy system; there can be one or more instances of the same energy system
energy_system_instance.aggrType = esdl.AggrTypeEnum.PER_COMMODITY
energy_system.instance.append(energy_system_instance)

#Every energy system has an area
energy_system.instance[0].area = esdl.Area(name="Municipality area")

#Create a PV park
pv_park = esdl.PVPark(name="PV park")
now = datetime.now()
pv_park.commissioningDate = now

#Create a household consumer  
el_demand = esdl.ElectricityDemand(name="Municipality")

#As in this case, both the PV park and the consumer are parts of the electricity network, an electricity network asset is created
el_network = esdl.ElectricityNetwork(name="Electricity Network")


energy_system.instance[0].area.asset.append(pv_park)
energy_system.instance[0].area.asset.append(el_demand)
energy_system.instance[0].area.asset.append(el_network)

### 1.2 Creating connections between the assets 

The second step in creating an ESDL energy system is to connect all the <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergyAsset.html">Energy Assets</a> in the network. Every <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergyAsset.html">Energy Asset</a> has an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/InPort.html">input (InPort)</a> and/or an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/OutPort.html">output (OutPort)</a> port associated with it. <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Asset.html">Assets</a> are connected through ports. In this tutorial, the PV park is the producer of electricity, therefore it has an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/OutPort.html">output port (OutPort)</a> associated with it, and the municipality consumes the electricity in the electricity network, therefore it has an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/InPort.html">input port (InPort)</a> associated with it. To connect the municipality consumer and the PV park producer, the <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/OutPort.html">output port</a> of the <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/PVPark.html">PVPark</a> asset is connected to the <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/InPort.html">input port</a> of the <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/ElectricityDemand.html">ElectricityDemand</a> asset. The order can also be reversed.

<img src="images/assets_io_ports.png">

In [2]:
in_port = esdl.InPort(id='InPort_ED')
el_demand.port.append(in_port)

out_port = esdl.OutPort(id='Out_Port_PV_Park', connectedTo=[in_port])
pv_park.port.append(out_port)

### 1.3 Creating energy carriers or commodities

The third step when creating an ESDL energy system is to add <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Carriers.html">Carriers</a> and/or <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Commodity.html">Commodities</a>. <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Carriers.html">Carriers</a> and <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Commodity.html">Commodities</a> describe what type of energy is carried through the network. They add extra <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystemInformation.html">EnergySystemInformation</a>, and depending on the model and the level of detail, they can be used to add emissions, energy content, associated measuring units etc. <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Carriers.html">Carriers</a> are associated to the <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Port.html">Ports</a>.

In this tutorial, the only commodity is <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/ElectricityCommodity.html">ElectricityCommodity</a>.

In [3]:
# An energy system has some information that can be used accross the energy system, for example, the types of carriers in the system
# Since this example only models electricity consumers and prosumers, the only carrier specified for now should be electricity

electricity_commodity = esdl.ElectricityCommodity(name="Electricity")
energy_system_information = esdl.EnergySystemInformation(id=str(uuid.uuid4()))
carriers = esdl.Carriers(id=str(uuid.uuid4()))
carriers.carrier.append(electricity_commodity)

energy_system.energySystemInformation = energy_system_information
energy_system.energySystemInformation.carriers = carriers

in_port.carrier = electricity_commodity
out_port.carrier = electricity_commodity

### 1.4 Creating supply and demand profiles

The last step in creating an ESDL <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a> model is to add production and demand <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Profiles.html">Profiles</a> to <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergyAsset.html">Energy Assets</a> (the PV park and the municipality consumer, respectively). This tutorial uses <a href="https://edr.hesi.energy/">Energy Data Repository (EDR)</a> to get and build the profiles. <a href="https://edr.hesi.energy/">EDR</a> is a prototype of a publicly accessible database system with open energy data. Examples are models of energy assets (with powers, efficiency, cost information), typical profiles for different consumers and producers, or information about energy carriers (with energy content and emissions). All the data from <a href="https://edr.hesi.energy/">EDR</a> is available in ESDL. The data can be accessed using <a href="https://edr.hesi.energy/api">EDR API</a> (and retrieved from InfluxDB, a timeseries database), or downloaded as an ESDL file directly from the user interface.



The following code hides the implementation details of connecting to and retrieving data from EDR API and InfluxDB. Those interested in the code details can go through edrreader.py and influxdbreader.py.

As is the case with <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Carriers.html">Carriers</a>, <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/Profiles.html">Profiles</a> are also associated to ports.

<img src="images/assets_profiles.png">

In [4]:
import influxdbreader as idb
import pandas as pd 
import numpy as np

# Reads the yearly solar production
yearly_solar_production = idb.read_yearly_solar_production_profile_data(2015)

# As this tutorial models a PV park as a producer, the data for an average PV park is taken from Energy Data Repository specification
# https://edr.hesi.energy/
# TODO: Check -> According to EDR specification, an average/typical (????) PV park produces 20000 kW and gives 867 full load hours 
yearly_solar_production['Production [kW]'] = yearly_solar_production['value'] * 17340000

In [5]:
# Reads the yearly demand profile for an average household
yearly_el_demand = idb.read_yearly_demand_profile_data(2015)

# In this example, an example of an area with 580 households (of a family with 1 child) is modelled
# An average Dutch household (a family with a child) uses 3500kWh per year 
# For an area with 580 households, this amounts to approximately 2 TWh
# Scale to kWh
yearly_el_demand['E1A consumption [kWh]'] = yearly_el_demand['E1A'] * 2030000

There are multiple ways to store a profile in ESDL, for example, as an <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/InfluxDBProfile.html">InfluxDBProfile</a> or as a <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/DateTimeProfile.html">DateTimeProfile</a>. In this example, profiles are stored as <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/DateTimeProfile.html">DateTimeProfile</a> to show it in ESDL. However, this creates an ESDL element for every profile value, which can lead to a very long ESDL file. Therefore, it is recommended to store your profiles in an InfluxDB using <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/InfluxDBProfile.html">InfluxDBProfile</a>.

For the purpose of this tutorial, a <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/DateTimeProfile.html">DateTimeProfile</a> is created for 24 hours (January 1st 2015).

In [6]:
# For every hour, a ProfileElement is created and stored into DateTimeProfile

# Fix python builtin 'from' that is also used in ProfileElement as attribute
esdl.ProfileElement.from_.name = 'from' 
setattr(esdl.ProfileElement, 'from', esdl.ProfileElement.from_)

# Add  solar production profile to the PV parc
# A new DateTimeProfile object is created to hold the full production profile
solar_production_datetime_profile = esdl.DateTimeProfile()

daily_prod_profile = yearly_solar_production[0:24]

demand_profile_iterator = daily_prod_profile.iterrows()

# Each hour is stored as a ProfileElement and added to the PV park's DateTimeProfile
for index, row in demand_profile_iterator:
    edate = datetime.strptime(index.strftime("%Y-%m-%dT%H:%M:%S"), "%Y-%m-%dT%H:%M:%S")    
    profile_element = esdl.ProfileElement(from_ = edate, to = edate + pd.DateOffset(hours=1), value = row['Production [kW]']) #from = ,
    solar_production_datetime_profile.element.append(profile_element)

# As the profiles are associated with ports, and the PV park (producer) has an OutPort attached to it, the created DateTimeProfile is added to the out port 
out_port.profile.append(solar_production_datetime_profile)


In [7]:
# Add electricity load demand profle to the municipality consumer and store it to ESDL

# A new DateTimeProfile object is created to hold the full demand profile
load_demand_datetime_profile = esdl.DateTimeProfile()

# The profile is a 15-min profile; take 24 hours 
daily_load_profile = yearly_el_demand[0:96]

demand_profile_iterator = daily_load_profile.iterrows()

# Each hour is stored as a ProfileElement and added to the Municipality consumer's DateTimeProfile
for index, row in demand_profile_iterator:
    edate = datetime.strptime(index.strftime("%Y-%m-%dT%H:%M:%S"), "%Y-%m-%dT%H:%M:%S")    
    profile_element = esdl.ProfileElement(from_ = edate, to = edate + pd.DateOffset(minutes=15), value = row['E1A consumption [kWh]']) #from = ,
    load_demand_datetime_profile.element.append(profile_element)

# As the profiles are associated with ports, and the consumer (ElectricityDemand) has an InPort attached to it, the created DateTimeProfile is added to the in port 
in_port.profile.append(load_demand_datetime_profile)

### 1.5 Saving a created energy system as an ESDL file

In [8]:
resource = rset.create_resource(URI('NL_ES_Model.esdl'))
resource.append(energy_system)
resource.save()

Now that the file is saved, we can explore its structure, add data, and further work with it. Below is the screenshot of the ESDL structure and two links to download the ESDL file and the Jupyter Notebook of this tutorial.

<img src="images/xml_esdl.png">

<a href="NL_ES_Model.esdl">Download here the ESDL file of the energy system</a>

<a href="ESDLTutorials.ipynb">Download here the ESDL tutorial Jupyter Notebook</a>

### 2 Loading an Energy System from an existing ESDL file

As seen in the previous part of the tutorial, one of the ways an ESDL <a href="https://energytransition.github.io/#router/doc-content/687474703a2f2f7777772e746e6f2e6e6c2f6573646c/EnergySystem.html">EnergySystem</a> can be created is from scratch through code. The other approach is to load an existing ESDL file of an energy system, and then further worked with. The next section loads the ESDL file created in the previous part of the tutorial, changes an attribute, and saves the changes.

In [9]:
rset_existing = ResourceSet()

resource_existing = rset_existing.get_resource(URI("NL_ES_Model.esdl"))
created_energy_system = resource_existing.contents[0]

# Energy system's attributes can be viewed, added or changed
print("Original name: " + created_energy_system.name)
created_energy_system.name = 'Energy System, The Netherlands'
print("New name: " + created_energy_system.name)

# The changes are saved in the existing ESDL file
resource_existing.save()

Original name: Nederland ES
New name: Energy System, The Netherlands
